In [ ]:
!pip install opencv-python numpy matplotlib ipywidgets Pillow


In [ ]:
# --------------------------------------------------------
# 1. Import all necessary libraries
# --------------------------------------------------------
import cv2                      # OpenCV for image reading, processing, enhancement
import numpy as np              # NumPy for numerical operations, LUTs, intensity arrays
import matplotlib.pyplot as plt # Matplotlib for plotting images, histograms, curves
import ipywidgets as widgets    # ipywidgets for interactive UI controls in Jupyter
from ipywidgets import HBox, VBox, Layout # For arranging widgets in dashboard layout
from IPython.display import display, clear_output # To display UI and update outputs dynamically
from PIL import Image           # Pillow library for image handling and conversions

# --------------------------------------------------------
# 2. Define plotting function
# --------------------------------------------------------


def plot_results(original_img, processed_img, transform_lut=None,
                 original_title="Original Image", processed_title="Processed Image"):
    """
    Shows results of image processing in 3 parts:
    1. Side-by-side comparison of original vs processed images
    2. Transformation function s = T(r) (if a LUT is provided)
    3. Histograms of original vs processed image for intensity distribution analysis
    """

    # --- Show Original and Processed Images ---
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))  # Two side-by-side plots

    # Left: Original image
    axes[0].imshow(original_img, cmap='gray', vmin=0,
                   vmax=255)  # show grayscale image
    axes[0].set_title(original_title)  # add title
    axes[0].axis('off')                # hide axis for cleaner look

    # Right: Processed image (after enhancement)
    axes[1].imshow(processed_img, cmap='gray', vmin=0, vmax=255)
    axes[1].set_title(processed_title)
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()

    # --- Show Transformation Function (if LUT provided) ---
    # Example: Gamma curve, Contrast Stretching mapping, etc.
    if transform_lut is not None:
        plt.figure(figsize=(5, 4))
        input_intensities = np.arange(256)    # 0–255 grayscale input levels
        plt.plot(input_intensities, transform_lut)  # map input → output
        plt.title('Transformation Function s = T(r)')
        plt.xlabel('Input Intensity (r)')
        plt.ylabel('Output Intensity (s)')
        plt.xlim([0, 255])
        plt.ylim([0, 255])
        plt.grid(True, linestyle='--')
        plt.show()

    # --- Show Histograms ---
    # Histograms prove how enhancement affects intensity distribution
    fig, axes = plt.subplots(1, 2, figsize=(10, 4))

    # Original histogram (blue)
    axes[0].hist(original_img.ravel(), bins=256, range=(0, 255), color='blue')
    axes[0].set_title("Original Histogram")
    axes[0].set_xlim([0, 255])

    # Processed histogram (green)
    axes[1].hist(processed_img.ravel(), bins=256,
                 range=(0, 255), color='green')
    axes[1].set_title("Processed Histogram")
    axes[1].set_xlim([0, 255])

    plt.tight_layout()
    plt.show()
# --------------------------------------------------------
# --------------------------------------------------------
# --- Layouts ---
# --------------------------------------------------------


# Box layout style for widget borders/padding/margin
box_layout = Layout(border='solid 1px #cccccc', padding='10px', margin='5px')

# Simple CSS styling for titles (HTML headings)
title_style = '<style>h2 { margin: 0; padding: 0; } h3 { margin: 5px 0; padding: 0; color: #555;}</style>'


# --------------------------------------------------------
# --- Main Title and File Uploader ---
# --------------------------------------------------------

# Dashboard title displayed at the top
dashboard_title = widgets.HTML(
    f"<h2>{title_style}Chest X-Ray Enhancement Dashboard</h2>")

# File uploader widget, restricted to images only
# "description" sets button label, layout sets width
uploader = widgets.FileUpload(
    accept='image/*', description='Upload Image', layout={'width': 'max-content'})


# --------------------------------------------------------
# --- Output Area ---
# --------------------------------------------------------

# Output panel where processed results (images, histograms, etc.) will appear
output_area = widgets.Output(
    layout=Layout(border='solid 1px #cccccc', padding='10px', width='65%')
)

# Title for the output section
output_title = widgets.HTML(f"<h3>{title_style}Output</h3>")


# --------------------------------------------------------
# --- Operation Widgets (Buttons + Sliders) ---
# --------------------------------------------------------

# Buttons for each enhancement technique
gamma_button = widgets.Button(description="Apply Gamma", button_style='info')
hist_button = widgets.Button(
    description="Apply Histogram Equalization", button_style='info')
contrast_button = widgets.Button(
    description="Apply Contrast Stretching", button_style='info')
combo_button = widgets.Button(
    description="Apply Gamma + HistEq", button_style='success')  # combo of two
# --- Save Button ---
save_button = widgets.Button(description="Save Enhanced Image", button_style='warning')

# Sliders to adjust parameters dynamically
gamma_slider = widgets.FloatSlider(
    value=1.0, min=0.1, max=5.0, step=0.1, description='Gamma:')
contrast_slider = widgets.IntRangeSlider(
    value=[50, 200], min=0, max=255, step=1, description='Stretch Range:')


# --------------------------------------------------------
# --- Global variable to store uploaded image ---
# --------------------------------------------------------
# stores uploaded X-ray as grayscale (for re-use across functions)
original_image = None
# Variable to store the last processed image
last_processed_image = None

# --------------------------------------------------------
# --- Event Handlers ---
# --------------------------------------------------------

# File uploader event: called when a user uploads an image
def on_upload_change(change):
    global original_image
    if not uploader.value:
        return  # skip if no file uploaded

    # Extract uploaded file content
    filename = next(iter(uploader.value))             # get filename
    file_info = uploader.value[filename]              # get file info
    img_bytes = file_info['content']                  # extract raw bytes
    nparr = np.frombuffer(img_bytes, np.uint8)        # convert to numpy array
    # decode into grayscale image
    original_image = cv2.imdecode(nparr, cv2.IMREAD_GRAYSCALE)

    # Reset uploader (so user can re-upload same file if needed)
    uploader.value.clear()

    # Display uploaded image in output area
    with output_area:
        clear_output(wait=True)
        display(widgets.HTML("<h4>Original Image Uploaded:</h4>"))
        display(Image.fromarray(original_image))


# Helper function to process image and show results
def process_and_plot(processed_image, title, transform_lut=None):
    global last_processed_image
    if original_image is None:
        with output_area:
            clear_output(wait=True)
            display(widgets.HTML("<p style='color:red;'>Error: Please upload an image first.</p>"))
        return

    last_processed_image = processed_image  # store latest result

    with output_area:
        clear_output(wait=True)
        plot_results(original_image, processed_image, transform_lut, processed_title=title)
        display(save_button)  # Show save option after enhancement

# --------------------------------------------------------
# --- Image Enhancement Technique Functions ---
# --------------------------------------------------------

# 1. Gamma Transformation (Power-Law Transformation)
# Formula: s = c * r^γ
#   r = input pixel (normalized between 0 and 1)
#   s = output pixel
#   γ = gamma value (set using the slider)
#   c = scaling constant (usually 1)
#
# How it works:
# - If γ < 1: dark regions brighten (good for X-rays where lungs are too dark).
# - If γ > 1: bright regions get suppressed (helps if white dominates).
# - If γ = 1: no change.
#
# Implementation: build a lookup table (LUT) where each input intensity (0–255)
# is mapped to a new output value using the gamma formula.


def on_gamma_button_clicked(b):
    if original_image is None:   # SAFEGUARD
        with output_area:
            clear_output(wait=True)
            display(widgets.HTML("<p style='color:red;'>Please upload an image first.</p>"))
        return
    gamma = gamma_slider.value
    lut = np.array([((i / 255.0) ** gamma) * 255 for i in range(256)]).astype("uint8") #LUT formula
    processed = cv2.LUT(original_image, lut)  # Apply LUT to image
    process_and_plot(processed, f"Gamma (γ={gamma:.2f})", lut)
    with output_area:
      display(widgets.HTML(f"""
          <p><b>Transformation Function:</b> s = c · r<sup>γ</sup><br>
          where c = 1, γ = {gamma:.2f}</p>
          <p><i>Interpretation:</i> γ < 1 brightens dark regions, γ > 1 darkens bright regions.</p>
      """))


# 2. Histogram Equalization (HE)
# Idea: spread out pixel intensities across the full range [0, 255].
#
# Steps:
# 1. Compute histogram h(rk).
# 2. Normalize to probability p(rk) = h(rk)/(M*N).
# 3. Compute cumulative distribution function (CDF).
# 4. Map intensities: sk = (L - 1) * c(rk), where L=256 for 8-bit images.
#
# Effect:
# - Dark values get lifted, bright values compressed.
# - Enhances global contrast (details become more visible).
#
# OpenCV provides cv2.equalizeHist(img), which performs all the steps.

def on_hist_button_clicked(b):
    if original_image is None:   # SAFEGUARD
        with output_area:
            clear_output(wait=True)
            display(widgets.HTML("<p style='color:red;'>Please upload an image first.</p>"))
        return

    # --- SAFEGUARD ---
    if len(original_image.shape) == 3:  # If uploaded as color
        gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
    else:
        gray = original_image
    # -----------------

    processed = cv2.equalizeHist(gray)  # OpenCV built-in HE
    process_and_plot(processed, "Histogram Equalization")
    with output_area:
      display(widgets.HTML("""
          <p><b>Transformation Function:</b> s<sub>k</sub> = (L-1) · Σ p(r<sub>j</sub>)<br>
          where L = 256 (gray levels), p(r<sub>j</sub>) = probability of intensity.</p>
          <p><i>Interpretation:</i> Redistributes intensities for better global contrast.</p>
      """))


# 3. Contrast Stretching (Piecewise Linear Transformation)
# Formula:
#   For chosen range [r_min, r_max] mapped to [0, 255]:
#   s = ((r - r_min) / (r_max - r_min)) * 255
#
# How it works:
# - Intensities below r_min → 0 (black).
# - Intensities above r_max → 255 (white).
# - Values in between are linearly stretched.
#
# Example:
# - If intensities originally between 50–200, after stretching:
#   50 → 0, 200 → 255, everything else spread out.

def on_contrast_button_clicked(b):
    if original_image is None:   # SAFEGUARD if img not uploaded
        with output_area:
            clear_output(wait=True)
            display(widgets.HTML("<p style='color:red;'>Please upload an image first.</p>"))
        return

    low, high = contrast_slider.value
    # --- SAFEGUARD ---
    if high <= low:   # avoid division by zero or invalid range
        with output_area:
            clear_output(wait=True)
            display(widgets.HTML(
                "<p style='color:red;'>Invalid range: please ensure High > Low.</p>"))
        return
    # -----------------

    lut = np.zeros(256, dtype=np.uint8)
    # map low..high to 0..255
    lut[low:high] = np.linspace(0, 255, high - low).astype("uint8")
    lut[high:] = 255
    processed = cv2.LUT(original_image, lut)
    process_and_plot(processed, f"Contrast Stretching ({low}-{high})", lut)
    with output_area:
      display(widgets.HTML(f"""
          <p><b>Transformation Function:</b> s = (r - r<sub>min</sub>) / (r<sub>max</sub> - r<sub>min</sub>) · 255<br>
          where r<sub>min</sub> = {low}, r<sub>max</sub> = {high}</p>
          <p><i>Interpretation:</i> Expands intensities between {low}-{high} to full [0,255] range.</p>
      """))

# 4. Gamma + Histogram Equalization (Combination)
# Steps:
# - First apply Gamma Correction → enhances dark details (inside lungs).
# - Then apply Histogram Equalization → redistributes intensities for overall balance.
#
# Why this order:
# - Gamma reveals hidden dark details.
# - Histogram Equalization spreads those details across full [0–255] range.
def on_combo_button_clicked(b):
    if original_image is None:   # SAFEGUARD
        with output_area:
            clear_output(wait=True)
            display(widgets.HTML("<p style='color:red;'>Please upload an image first.</p>"))
        return
    gamma = gamma_slider.value
    lut = np.array([((i / 255.0) ** gamma) * 255 for i in range(256)]).astype("uint8") #LUT formula
    gamma_img = cv2.LUT(original_image, lut)       # Step 1: Gamma
    processed = cv2.equalizeHist(gamma_img)        # Step 2: HistEq
    process_and_plot(processed, f"Gamma (γ={gamma:.2f}) + Histogram Equalization")
    with output_area:
      display(widgets.HTML(f"""
          <p><b>Transformation Functions:</b><br>
          Step 1: s = c · r<sup>γ</sup>, with γ = {gamma:.2f}, c = 1<br>
          Step 2: Histogram Equalization → spreads intensities across [0,255]</p>
          <p><i>Interpretation:</i> Gamma reveals hidden dark details, HE balances overall contrast.</p>
      """))


# Save button handler
def on_save_button_clicked(b):
    global last_processed_image
    if last_processed_image is None:
        with output_area:
            clear_output(wait=True)
            display(widgets.HTML("<p style='color:red;'>Error: Please enhance an image first before saving.</p>"))
        return

    # Build filename with timestamp
    import datetime
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"enhanced_{timestamp}.png"

    # Save locally (works in both Colab and VS Code)
    cv2.imwrite(filename, last_processed_image)

    # Try Colab auto-download (won't break outside Colab)
    try:
        from google.colab import files
        files.download(filename)
        msg = f"Image saved and downloaded as <b>{filename}</b>"
    except ImportError:
        # If not in Colab, just save to working directory
        msg = f"Image saved locally as <b>{filename}</b> (check your project folder)."

    # Feedback in the dashboard
    with output_area:
        clear_output(wait=True)
        display(widgets.HTML(f"<p style='color:green;'>{msg}</p>"))

# --------------------------------------------------------
# --- Link Buttons to Their Handlers ---
# --------------------------------------------------------
uploader.observe(on_upload_change, names='value')
gamma_button.on_click(on_gamma_button_clicked)
hist_button.on_click(on_hist_button_clicked)
contrast_button.on_click(on_contrast_button_clicked)
combo_button.on_click(on_combo_button_clicked)
save_button.on_click(on_save_button_clicked)

# --------------------------------------------------------
# --- Accordion UI for Controls ---
# --------------------------------------------------------
accordion = widgets.Accordion(children=[
    VBox([gamma_slider, gamma_button]),   # Gamma
    hist_button,                          # Histogram Eq
    VBox([contrast_slider, contrast_button]),  # Contrast Stretching
    VBox([gamma_slider, combo_button])    # Combo
])
accordion.set_title(0, 'Power Law (Gamma)')
accordion.set_title(1, 'Histogram Equalization')
accordion.set_title(2, 'Contrast Stretching')
accordion.set_title(3, 'Gamma + HistEq (Combination)')


# --------------------------------------------------------
# --- Final Layout: Controls + Output ---
# --------------------------------------------------------
controls_panel = VBox([
    widgets.HTML(f"<h3>{title_style}Controls</h3>"),
    uploader,
    accordion
], layout=Layout(border='solid 1px #cccccc', padding='10px', width='35%'))

output_panel = VBox([output_title, output_area])
dashboard = HBox([controls_panel, output_panel])


# --------------------------------------------------------
# --- Display the Dashboard ---
# --------------------------------------------------------
display(dashboard_title, dashboard)

# ------------------- HISTOGRAM INTERPRETATION GUIDE -------------------
# 1. Gamma Transformation
#    - Effect: Curves the histogram depending on gamma (γ).
#        * γ < 1 → Brightens dark regions (histogram shifts right).
#        * γ > 1 → Darkens bright regions (histogram shifts left).
#    - Note: Histogram does NOT necessarily span 0–255. This is normal.

# 2. Histogram Equalization (HE)
#    - Effect: Redistributes pixel values to spread intensities more evenly.
#    - Histogram: Usually spans most of 0–255 and looks flatter/more uniform.
#    - Jagged shape is normal due to discrete pixel levels.

# 3. Contrast Stretching
#    - Effect: Maps input range [low, high] → [0, 255].
#        * Pixels < low → become 0.
#        * Pixels > high → become 255.
#    - Histogram: Expands to cover full 0–255.
#    - Expect spikes at 0 and 255 (clipping), with stretched middle region.

# 4. Gamma + Histogram Equalization (Combination)
#    - Effect: Gamma reveals hidden details by reshaping pixel distribution,
#              then HE spreads them across full intensity range.
#    - Histogram: Spans 0–255 like HE, but distribution is more balanced.
#    - Often gives the best detail + contrast enhancement.
# -----------------------------------------------------------------------

